In [ ]:
from dotenv import load_dotenv
# Load envionment variables
load_dotenv()

In [ ]:
# Load packages
import os

from codecarbon import EmissionsTracker
import mlflow
import mlflow.pytorch
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torch.optim as optim
from ptflops import get_model_complexity_info

from mdsist.architectures import CNN
from mdsist.dataset import MdsistDataset
from mdsist.config import PROCESSED_DATA_DIR, RAW_DATA_DIR, MODELS_DIR
from mdsist.trainer import Trainer

import random
import numpy as np

In [3]:
def seed_all(value: int) -> None:
    np.random.seed(value)
    torch.manual_seed(value)
    torch.cuda.manual_seed(value)  # Set seed for CUDA
    torch.cuda.manual_seed_all(value)  # Use this to ensure that all GPUs get the same seed
    random.seed(value)

    torch.backends.cudnn.deterministic = True  # Ensure deterministic behavior for cudnn
    torch.backends.cudnn.benchmark = False  # Disable cudnn's benchmark mode for reproducibility

In [4]:
# Set hyperparameters
EPOCHS = 5
BATCH_SIZE = 64
LEARNING_RATE = 0.0001
SEED = 42

In [5]:
# Seed for reproducibility
seed_all(42)

In [6]:
# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load datasets
train_dataset = MdsistDataset(PROCESSED_DATA_DIR / 'train.parquet', transform=transform)
val_dataset = MdsistDataset(PROCESSED_DATA_DIR / 'validation.parquet', transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
# Init model
model = CNN()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

flops, params = get_model_complexity_info(model, (1, 28, 28), as_strings=True, print_per_layer_stat=False, verbose=False)

print(f'FLOPS: {flops}')
print(f'Parameters: {params}')

In [8]:
# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Experiment ID
EXPERIMENT_ID = 'CNN_v2'
# Track emissions
# emissions_tracker = EmissionsTracker()
emissions_tracker = EmissionsTracker(project_name='MDSIST', experiment_id=EXPERIMENT_ID)

In [ ]:
emissions_tracker.start()

mlflow.set_tracking_uri(os.getenv('MLFLOW_TRACKING_URI'))
mlflow.set_experiment(EXPERIMENT_ID)

with mlflow.start_run():
    mlflow.set_tag('mlflow.runName', 'Train')

    # Log hyperparameters
    mlflow.log_param('epochs', EPOCHS)
    mlflow.log_param('batch_size', BATCH_SIZE)
    mlflow.log_param('learning_rate', LEARNING_RATE)
    mlflow.log_param('seed', SEED)
    
    # Train
    trainer = Trainer(model, optimizer, device)
    trainer.train(train_loader, val_loader, 5)

    # Log the model itself to MLflow
    mlflow.pytorch.log_model(trainer.model, "model")

    emissions = emissions_tracker.stop()

    mlflow.log_metric('emissions_kg_co2', emissions)

In [11]:
torch.save(trainer.model, MODELS_DIR / 'model.pt')